In [ ]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel

# import torch
import subprocess
from pprint import pprint
import os
from openai import OpenAI

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 讀檔案

In [ ]:
df = pd.read_csv(
    r"c:\Users\student\期末專題\爬蟲\Crawler_1140718\104_programmers_jobs.csv"
)



df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 46 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  15748 non-null  object 
 1   更新日期    15748 non-null  object 
 2   查詢職類    15748 non-null  int64  
 3   查詢關鍵字   15748 non-null  object 
 4   職務類別    15748 non-null  object 
 5   職缺名稱    15748 non-null  object 
 6   公司名稱    15748 non-null  object 
 7   公司連結    15748 non-null  object 
 8   產業別     15748 non-null  object 
 9   上班地點    15748 non-null  object 
 10  地區      14577 non-null  object 
 11  城市      15748 non-null  object 
 12  國家      15748 non-null  object 
 13  薪資      15748 non-null  object 
 14  薪資下限    15748 non-null  int64  
 15  薪資上限    15748 non-null  int64  
 16  職缺描述    15733 non-null  object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  15748 non-null  object 
 19  學歷要求    15748 non-null  object 
 20  科系要求    8534 non-null   object 
 21  擅長工具    11180 non-null  object 
 22

## ollama

In [ ]:
df = df.head(5).copy()

df["技能原文"] = (
    df["職缺描述"].astype(str)
    + "\n"
    + df["擅長工具"].astype(str)
    + "\n"
    + df["工作技能"].astype(str)
)

df.info()
df[["職缺連結", "職缺名稱", "技能原文"]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 49 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  5 non-null      object 
 1   更新日期    5 non-null      object 
 2   查詢職類    5 non-null      int64  
 3   查詢關鍵字   5 non-null      object 
 4   職務類別    5 non-null      object 
 5   職缺名稱    5 non-null      object 
 6   公司名稱    5 non-null      object 
 7   公司連結    5 non-null      object 
 8   產業別     5 non-null      object 
 9   上班地點    5 non-null      object 
 10  地區      5 non-null      object 
 11  城市      5 non-null      object 
 12  國家      5 non-null      object 
 13  薪資      5 non-null      object 
 14  薪資下限    5 non-null      int64  
 15  薪資上限    5 non-null      int64  
 16  職缺描述    5 non-null      object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  5 non-null      object 
 19  學歷要求    5 non-null      object 
 20  科系要求    1 non-null      object 
 21  擅長工具    4 non-null      object 
 22  工作技能  

,職缺連結,職缺名稱,技能原文
0,https://www.104.com.tw/job/8f54d,[R&D] Frontend Engineer 前端工程師,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
1,https://www.104.com.tw/job/7p2cn,[R&D] Senior Frontend Engineer 資深前端工程師,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
2,https://www.104.com.tw/job/8cbyv,後端工程主管,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...
3,https://www.104.com.tw/job/8b902,Senior Software Development Lead,"Yating(https://yating.tw/zh-hant), a Taiwan AI..."
4,https://www.104.com.tw/job/8bz2q,【擴編】資深前端工程師 Senior Frontend Developer,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...


In [ ]:
def query_ollama(prompt):
    result = subprocess.run(
        ["ollama", "run", "qwen:7b"], input=prompt.encode(), stdout=subprocess.PIPE
    )
    return result.stdout.decode()


def extract_skills(text):
    prompt = f"""

    請從以下內容中提取技能，列出技能或工具，用逗號分隔，
    內容中不要有其他文字
    (EX：不要有:以下是从提供的内容中提取的技能...等文字)
    ，單純回應技能名稱(如果有中英文版本給我中英文版本)
    (EX：HTML、CSS...等。)
    
    ：{text}

    """

    return query_ollama(prompt).strip()


# 新增提取技能欄位

In [43]:
df["技能"] = df["技能原文"].apply(extract_skills)
df.to_csv("output_with_skills.csv", index=False)

df_test = df[["技能", "技能原文"]]
df_test

,技能,技能原文
0,前端开发技能：HTML、CSS、JavaScript、React、Next.js、Vue、N...,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
1,前端开发技能：\n- HTML\n- CSS\n- JavaScript\n- TypeSc...,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
2,技能名称：\n\n1. 英文阅读撰寫能力（中级以上）\n2. Backend程式开发经验 (...,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...
3,技能列表：\n\n1. Java\n2. PHP\n3. Go\n4. JavaScript...,"Yating(https://yating.tw/zh-hant), a Taiwan AI..."
4,技能列表如下（以逗号分隔）：\n\n- HTML\n- CSS\n- JavaScript\...,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...


In [53]:
print(df_test["技能"].iloc[0])

前端开发技能：HTML、CSS、JavaScript、React、Next.js、Vue、Nuxt.js

WebXR相关技能：Three.js、TypeScript、WebGL、沉浸式体验开发经验

其他工具或语言：Git、Redux、Node.js、AWS、Github等。熟悉结构化程式设计和系统整合分析。


## GPT-API

In [ ]:
df = pd.read_csv(r"./104_programmers_jobs.csv")
df = df.head(5).copy()

df["技能原文"] = (
    df["職缺描述"].astype(str)
    + "\n"
    + df["擅長工具"].astype(str)
    + "\n"
    + df["工作技能"].astype(str)
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  5 non-null      object 
 1   更新日期    5 non-null      object 
 2   查詢職類    5 non-null      int64  
 3   查詢關鍵字   5 non-null      object 
 4   職務類別    5 non-null      object 
 5   職缺名稱    5 non-null      object 
 6   公司名稱    5 non-null      object 
 7   公司連結    5 non-null      object 
 8   產業別     5 non-null      object 
 9   上班地點    5 non-null      object 
 10  地區      5 non-null      object 
 11  城市      5 non-null      object 
 12  國家      5 non-null      object 
 13  薪資      5 non-null      object 
 14  薪資下限    5 non-null      int64  
 15  薪資上限    5 non-null      int64  
 16  職缺描述    5 non-null      object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  5 non-null      object 
 19  學歷要求    5 non-null      object 
 20  科系要求    1 non-null      object 
 21  擅長工具    4 non-null      object 
 22  工作技能  

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)


def extract_skills_gpt(text, model="gpt-4o"):
    prompt = f"""
你現在是一位資深的軟體工程師，熟悉市面上全部跟軟體相關的技能。也擅長讀懂職缺JD裡面的內容，並分析出職缺所需要的技能。
請從以下內容中提取技能，列出技能或工具，用逗號分隔。

請遵守以下規則：
1. 不要包含任何軟實力（如：溝通能力、團隊合作）
2. 僅列出技能名稱（例如：HTML、CSS、Python）
3. 使用繁體中文回覆
4. 去除與技能無關的文字
5. 回覆應僅包含技能清單（例：HTML,CSS,Python），不需其他解釋

以下是職缺內容：
{text}
"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"錯誤：{e}"


# ✅ 套用 GPT 抽取技能（你可以先測試前幾筆）
df["技能"] = df["技能原文"].astype(str).apply(extract_skills_gpt)
df.to_csv("output_skills.csv", index=False, encoding="utf-8-sig")

In [19]:
df_test = df[["技能", "技能原文"]]
df_test

,技能,技能原文
0,"HTML,CSS,JavaScript,React,Next.js,Vue,Nuxt.js,...",我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
1,"HTML,CSS,JavaScript,TypeScript,React,Next.js,V...",我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
2,"Node.js,JavaScript,ES6,TypeScript,MongoDB,Redi...",工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...
3,"TypeScript,JavaScript,Python,Go,PHP,Java,Linux...","Yating(https://yating.tw/zh-hant), a Taiwan AI..."
4,"HTML,CSS,JavaScript,TypeScript,Vue",主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...


In [ ]:
# 先將技能欄位字串拆分成列表
df["技能清單"] = (
    df["技能"]
    .astype(str)
    .apply(lambda x: [s.strip() for s in x.split(",") if s.strip()])
)

# 將每一列的技能列表展開為多欄
skills_expanded = df["技能清單"].apply(pd.Series)

# 將欄位重新命名為 技能1, 技能2, ...
skills_expanded.columns = [f"技能{i+1}" for i in skills_expanded.columns]

# 合併回原本的 df
df_final = pd.concat([df, skills_expanded], axis=1)

df_final.to_csv("output_skills.csv", index=False, encoding="utf-8-sig")
df_final.filter(regex="^技能")

,技能原文,技能,技能清單,技能1,技能2,技能3,技能4,技能5,技能6,技能7,...,技能11,技能12,技能13,技能14,技能15,技能16,技能17,技能18,技能19,技能20
0,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...,"HTML,CSS,JavaScript,React,Next.js,Vue,Nuxt.js,...","[HTML, CSS, JavaScript, React, Next.js, Vue, N...",HTML,CSS,JavaScript,React,Next.js,Vue,Nuxt.js,...,TypeScript,WebGL,GIT,Redux,VueJS,ReactJS,Node.js,AWS,Github,NaN
1,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...,"HTML,CSS,JavaScript,TypeScript,React,Next.js,V...","[HTML, CSS, JavaScript, TypeScript, React, Nex...",HTML,CSS,JavaScript,TypeScript,React,Next.js,Vue,...,Three.js,WebGL,Git,AWS,Node.js,ReactJS,VueJS,Redux,NaN,NaN
2,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...,"Node.js,JavaScript,ES6,TypeScript,MongoDB,Redi...","[Node.js, JavaScript, ES6, TypeScript, MongoDB...",Node.js,JavaScript,ES6,TypeScript,MongoDB,Redis,Docker,...,https,websocket,tcp/ip,DevOps,AWS,GCP,Digital Ocean,System monitoring,Github,Git
3,"Yating(https://yating.tw/zh-hant), a Taiwan AI...","TypeScript,JavaScript,Python,Go,PHP,Java,Linux...","[TypeScript, JavaScript, Python, Go, PHP, Java...",TypeScript,JavaScript,Python,Go,PHP,Java,Linux,...,算法,資料結構,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...,"Vue,HTML,CSS,JavaScript,TypeScript","[Vue, HTML, CSS, JavaScript, TypeScript]",Vue,HTML,CSS,JavaScript,TypeScript,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Skill_list = "C11,C++17-STL,Objective-C,Swift-Concurrency,Go-Generics,Rust-Tokio,Kotlin-KMP,Java-Spring Boot,Groovy-Gradle,Scala-Akka,Python-3.12,Python-Typing,TypeScript-5.x,JavaScript-ES2023,Dart,Ruby-on-Rails,PHP-Laravel,Elixir-Phoenix,Haskell-GHC,R-Tidyverse,Julia-DataFrames,MATLAB-Simulink,Lua-OpenResty,Bash-Shell,PowerShell-Core,Perl-Moose,Assembly-x86-64,WebAssembly-WASI,SQL-ANSI,GraphQL-SDL,JSON-Schema,YAML,TOML,Protocol Buffers,Thrift-IDL,Regular Expression-PCRE,Makefile,CMake,Gradle-Kotlin DSL,Maven-POM,NPM-Workspaces,Yarn-Plug’n’Play,PNPM,Pip-Requirements,Poetry-PyProject,Conda-Env,Cargo-Crates,Go-Modules,Swift-Package Manager,.NET-CLI,Git-Flow,Git-Submodule,Git-Worktree,Git-LFS,SVN,Mercurial,SSH-Keygen,GPG-Sign,Regex101-Testing,Postman-Collections,Insomnia-REST,HTTPie,cURL,JQ-JSON CLI,GNU-AWK,Sed-Stream,Fzf-Search,Tmux,Vim-Lua Config,Neovim-LSP,VSCode-Dev Container,JetBrains-IntelliJ,Eclipse,Emacs-Org Mode,Sublime-Build System,Notion-Markdown,Obsidian-Dataview,Docusaurus,MkDocs-Material,Sphinx-Autodoc,HTML-5,CSS-Flexbox,CSS-Grid,SCSS-Mixin,TailwindCSS,PostCSS,Styled-Components,React-Hooks,React-Redux Toolkit,React-Next.js 14,React-Remix,Vue3-Composition API,Nuxt-3,Angular-16,Svelte-Kit,SolidJS,Lit-Element,Webpack-Module Federation,Vite-HMR,Rollup-Treeshake,Parcel-Bundler,Storybook-CSF,Framer-Motion,Next-Auth,React-TanStack Query,SWR-Cache,React-React-Hook-Form,Apollo-Client,URQL,GraphQL-Codegen,D3.js,Chart.js,ECharts,Highcharts,Three.js,Babylon.js,WebGL-Shader,WebGPU,WebXR-AR,WebXR-VR,Service Worker-PWA,Workbox,WebRTC-Peer,WebSocket-Socket.IO,MQTT-Web,OAuth2-PKCE Flow,OpenID-Connect,Figma-Design Tokens,Adobe XD,Sketch-Symbol,Zeplin-Handoff,CSS-BEM,CSS-Modules,ESLint-Prettier,Husky-Git Hooks,Cypress-E2E,Playwright-Test,Lighthouse-Perf,Web-Vitals,Sentry-Browser SDK,Android-Kotlin,Android-Jetpack Compose,Android-Room DB,Android-CameraX,iOS-SwiftUI,iOS-Core Data,iOS-ARKit,Flutter-Riverpod,Flutter-Bloc,React Native-Expo,React Native-Hermes,Ionic-Capacitor,Cordova-Plugins,Unity-C#,Unreal-Blueprint,MAUI-.NET,Tauri-Rust Desktop,Electron-IPC,Expo-EAS Build,Kotlin Multiplatform,Xamarin-Forms,NativeScript-Vue,Fastlane-CI/CD,Firebase-Crashlytics,Firebase-Cloud Messaging,Firebase-Auth,Google Analytics-GA4,App Store Connect-TestFlight,Play Console-AB Testing,Lottie-Animation,ARCore-Depth API,ML-Kit-Vision,Mapbox-GL Native,OpenCV-Mobile,Realm-Mobile DB,Flutter-FFI,Skia-Canvas,React Native-Reanimated,Capacitor-Push,Expo-Router,Node.js-Express,Node.js-NestJS,FastAPI-Pydantic,Django-Rest Framework,Flask-Blueprint,Go-Fiber,Go-Echo,Rust-Axum,Rust-Actix-Web,.NET Core-Minimal API,Spring Cloud-Gateway,Micronaut-DI,Quarkus-Panache,gRPC-Proto Buf,gRPC-Gateway,GraphQL-Apollo Server,GraphQL-Hasura,REST-OpenAPI 3,JSON-API Spec,WebSocket-WS,MQTT-Mosquitto,NATS-JetStream,Kafka-Streams,RabbitMQ-AMQP,Redis-PubSub,OpenTelemetry-Trace,Jaeger-Collector,Zipkin-Brave,Kong-API Gateway,Envoy-Proxy,Istio-Service Mesh,Linkerd-Sidecar,Cadence-Workflow,Temporal-Workflow,Netflix-Hystrix,Resilience4j-Circuit,Polly-.NET Retry,Swagger-UI,PostgREST,Kratos-Identity,Ory-Hydra,Keycloak-OIDC,OAuth2-JWT,JWT-Auth0,bcrypt-Password Hash,Argon2-Hash,SAML-SSO,LDAP-Directory,OpenResty-Lua,Caddy-Server,H2-HTTP/2 Server,Fastify-Schema,Deno-Fresh,Bun-Hypercore,Finagle-RPC,ServiceComb-Java Chassis,Apache Thrift-RPC,ZeroMQ-Brokerless,Tarantool-Lua App Server,Orleans-Virtual Actor,Docker-Compose,Docker-BuildKit,Docker-Slim,Kubernetes-Helm Chart,Kubernetes-Kustomize,Kubernetes-Argo CD,Kubernetes-Argo Rollouts,Kubernetes-Flux,Terraform-AWS,Terraform-GCP,Terraform-Azure,Pulumi-TypeScript,Ansible-Playbook,Chef-Cookbook,SaltStack-States,Jenkins-Pipeline,Jenkins-Shared Lib,GitHub Actions-Matrix,GitLab CI-Runner,CircleCI-Orbs,Travis CI,Bamboo-Plan,Argo Workflows,Tekton-Pipelines,DroneCI-YAML,Spinnaker-Deployment,Harness-CD,AWS-CodePipeline,GCP-Cloud Build,Azure-DevOps Pipeline,HashiCorp-Vault,Consul-Service Discovery,Packer-Image Build,Prometheus-Alertmanager,Grafana-Loki,Grafana-Tempo,Elastic-Filebeat,Elastic-APM,Datadog-APM,New Relic-Insights,Splunk-Observability,OpenSearch-Dashboards,Sentry-Error Tracking,PagerDuty-Incident,OpsGenie-Alert,Grafana-OnCall,CloudWatch-Logs,CloudTrail-Audit,Azure-Monitor,GCP-Stackdriver,Falco-Runtime Security,Trivy-Image Scan,Anchore-Grype,OSQuery-Fleet,Kube-Bench,Kube-Hunter,SonarQube-Static Scan,FOSSA-License Scan,Renovate-Dependency Bot,Dependabot-Security PR,MySQL-InnoDB,MySQL-Group Replication,PostgreSQL-PostGIS,SQLite-Fts5,MariaDB-Galera,SQL Server-Always On,Oracle-RAC,MongoDB-Aggregation,Redis-Cluster,Redis-Sentinel,Memcached,Cassandra-CQL,DynamoDB-Global Table,Cosmos DB-SQL API,ScyllaDB,Couchbase-N1QL,Firestore-Document,Neo4j-Cypher,JanusGraph,ArangoDB-AQL,FaunaDB,Elasticsearch-Kibana Lens,Solr-Lucene,OpenSearch-Dashboards,MeiliSearch,Typesense,ClickHouse-MergeTree,DuckDB-Embedded OLAP,Snowflake-Time Travel,BigQuery-Partitioning,Redshift-Spectrum,Teradata-TD SQL,Vertica-EE,Hive-HQL,HBase-Phoenix,TiDB-HTAP,OceanBase,TimescaleDB-Hypertable,InfluxDB-Flux,VictoriaMetrics,CrateDB,Pinot-Realtime OLAP,Apache Druid,Hazelcast-IMDG,Aerospike-XDR,Tarantool-Cache,VoltDB-Fast ACID,CouchDB-MapReduce,LevelDB,RocksDB,Hadoop-HDFS,Spark-DataFrame API,Spark-Structured Streaming,Apache Flink-CEP,Apache Beam-Dataflow,Kafka-Connect,Kafka-KSQL DB,Storm-Topology,Samza,Pulsar-Functions,NiFi-Flow,Debezium-CDC,Sqoop,Airflow-DAG,Prefect-Flows,Dagster-Ops,Luigi-Pipeline,dbt-Core,GreatExpectations-Data Tests,DataHub-Metadata,Amundsen-Data Catalog,Trino-SQL Query,Presto-Distributed SQL,Apache Iceberg,Apache Hudi,Delta Lake,MinIO-S3,Alluxio-Cache,JuiceFS-POSIX,Ozone-Object Store,Parquet-Columnar,Avro-Schema,ORC-File,Feather-Arrow IPC,Arrow-Flight,FeatherV2,MLflow-Tracking,Feast-Feature Store,Tecton-Feature Platform,Metaflow-ML Pipeline,Kubeflow-Pipelines,Sagemaker-SDK,Azure-ML Studio,Vertex AI-Pipelines,DataRobot-AutoML,H2O-Driverless AI,Alteryx-Designer,KNIME-Analytics,RapidMiner-Studio,Talend-Data Fabric,Informatica-IICS,Pentaho-Kettle,Glue-ETL,AWS-DataBrew,BigQuery-Data Transfer,Databricks-Delta Live,Snowflake-Snowpipe,Redpanda-Streaming,Confluent-Schema Registry,Lenses.io-Kafka Ops,PyTorch-Lightning,TensorFlow-Keras,ONNX-Runtime,scikit-learn,XGBoost,LightGBM,CatBoost,FastAI-Tabular,HuggingFace-Transformers,Sentence-Transformers,LangChain-RAG,Llama-cpp,Alpaca-LoRA,QLoRA-Fine-Tune,PEFT-Adapter,BitsAndBytes-Quant,DeepSpeed-Zero3,Flash-Attention 2,Diffusers-Stable Diffusion,Audio-Whisper,Speech-NeMo,OpenAI-Embeddings,OpenAI-GPT-4o,Anthropic-Claude Opus,Google-Gemini 1.5,Mistral-Large,MiniLM-Sentence,GTE-Base-zh,E5-Multilingual,FAISS-IVF,Milvus-Vector DB,Pinecone-Index,Weaviate-Vectorize,Qdrant-Payload,Chroma-DB,Vespa-Search,Elastic-Vector,PgVector-Extension,Redis-Vector,OpenSearch-KNN,PowerBI-DAX,PowerBI-Paginated Reports,Tableau-Prep,Tableau-Hyper,Looker-LookML,Looker-Explore,Superset-SQL Lab,Metabase-Pulse,Grafana-Dashboard,Grafana-OnCall,Kibana-Canvas,Redash-Query,Qlik Sense,Amazon QuickSight,Google Data Studio,SAS-Visual Analytics,MicroStrategy-Dossiers,DOMO-Cards,FineBI,BOARD-Toolkit,Chartio-Reports,ClicData-Dash,Zoho-Analytics,Sisense-Fusion,Datapine-Predictive,Jupyter-Notebook,JupyterLab-Extensions,Observable-Plots,Vega-Lite,ECharts-Liquid,Plotly-Dash,Bokeh-Server,Altair-Grammar,Seaborn-Catplot,Matplotlib-Pyplot,ggplot2-R,Excel-Power Query,Google Sheets-App Script,Crystal Reports,JasperReports-Server"

In [ ]:
df = pd.read_csv(r"./104_programmers_jobs.csv")
df = df.head(5).copy()

df["技能原文"] = (
    df["職缺描述"].astype(str)
    + "\n"
    + df["擅長工具"].astype(str)
    + "\n"
    + df["工作技能"].astype(str)
)

df

,job_id,更新日期,查詢職類,查詢關鍵字,職務類別,職缺名稱,公司名稱,公司連結,產業別,上班地點,...,台語閱讀,台語寫作,其他條件,福利制度,法定福利,其他福利,面試流程,職缺連結,爬取時間,技能原文
0,8f54d,2025/06/30,2007001001,TypeScript,軟體工程師、前端工程師、軟體專案主管,[R&D] Frontend Engineer 前端工程師,iStaging愛實境_愛實境股份有限公司,https://www.104.com.tw/company/bjdang8,其它軟體及網路相關業,台北市松山區南京東路4段2號2樓（小巨蛋南區入口處）,...,NaN,NaN,NaN,員工認股、年終獎金、零食櫃、咖啡吧、國內旅遊、國外旅遊、部門聚餐、免費下午茶、優於勞基法特休...,週休二日、勞保、健保、特別休假、女性生理假、勞退提撥金,正職員工享有以下福利：\n\n【薪獎假勤類】\n. 具市場競爭力的薪資、年終獎金\n. 每月...,NaN,https://www.104.com.tw/job/8f54d,2025-07-18 00:13:38,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
1,7p2cn,2025/06/30,2007001001,TypeScript,軟體工程師、前端工程師、軟體專案主管,[R&D] Senior Frontend Engineer 資深前端工程師,iStaging愛實境_愛實境股份有限公司,https://www.104.com.tw/company/bjdang8,其它軟體及網路相關業,台北市松山區南京東路4段2號2樓（小巨蛋南區入口處）,...,NaN,NaN,NaN,員工認股、年終獎金、零食櫃、咖啡吧、國內旅遊、國外旅遊、部門聚餐、免費下午茶、優於勞基法特休...,週休二日、勞保、健保、特別休假、女性生理假、勞退提撥金,正職員工享有以下福利：\n\n【薪獎假勤類】\n. 具市場競爭力的薪資、年終獎金\n. 每月...,NaN,https://www.104.com.tw/job/7p2cn,2025-07-18 00:13:38,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...
2,8cbyv,2025/07/14,2007001001,TypeScript,後端工程師、軟體工程師、軟體專案主管,後端工程主管,香港商蒙卓有限公司台灣分公司,https://www.104.com.tw/company/1a2x6bmedx,網際網路相關業,台北市松山區南京東路五段99號5樓,...,NaN,NaN,▲ 加分項目 ▲\n- 有使用 Infrastructure as Code 經驗\n- 有...,年終獎金、零食櫃、不扣薪病假、下午茶、員工體檢假,NaN,我們重視每位員工，提供自主的工作流程，讓所有夥伴們能自行規劃工作時間與流程，認為工作與生活的...,NaN,https://www.104.com.tw/job/8cbyv,2025-07-18 00:13:39,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...
3,8b902,2025/07/09,2007001001,TypeScript,軟體工程師、軟體專案主管、演算法工程師,Senior Software Development Lead,Taiwan AI Labs_雅婷智慧股份有限公司,https://www.104.com.tw/company/1a2x6bm0oj,其它軟體及網路相關業,台北市大同區承德路1段70號15樓,...,NaN,NaN,NaN,NaN,NaN,正職福利制度\n\n▌彈性工時：正常工作時數內，每日1hr的彈性工時\n\n▌彈性遠端：每週...,NaN,https://www.104.com.tw/job/8b902,2025-07-18 00:13:40,"Yating(https://yating.tw/zh-hant), a Taiwan AI..."
4,8bz2q,2025/07/14,2007001001,TypeScript,前端工程師、軟體工程師、軟體專案主管,【擴編】資深前端工程師 Senior Frontend Developer,思量科技有限公司,https://www.104.com.tw/company/1a2x6bmxjc,電腦軟體服務業,台北市中正區忠孝東路二段26號2樓,...,NaN,NaN,NaN,年終獎金、三節獎金/禮品、零食櫃、不扣薪病假、不扣薪事假、部門聚餐、免費下午茶、優於勞基法特休,週休二日、勞保、健保、女性生理假,【勞基法的全方位保障，還有額外加碼！】 \n\n• 保障13個月年薪，讓工作與快樂同行！ \...,NaN,https://www.104.com.tw/job/8bz2q,2025-07-18 00:13:41,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...


In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)


def extract_skills_gpt(text, model="gpt-4o"):
    prompt = f"""
你現在是一位資深的軟體工程師，熟悉市面上全部跟軟體相關的技能。也擅長讀懂職缺JD裡面的內容，並分析出職缺所需要的技能。
請從以下內容中提取技能，列出技能或工具，用逗號分隔。
提取技能時優先參考 {Skill_list} 中的內容，
如果清單中沒有涵蓋到，就另外生成。

請遵守以下規則：
1. 不要包含任何軟實力（如：溝通能力、團隊合作）
2. 使用繁體中文回覆
3. 去除與技能無關的文字
4. 回覆應僅包含技能名稱（例：HTML,CSS,Python），不需其他解釋

以下是職缺內容：
{text}
"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"錯誤：{e}"


# ✅ 套用 GPT 抽取技能（你可以先測試前幾筆）
df["技能"] = df["技能原文"].astype(str).apply(extract_skills_gpt)
# df.to_csv("output_skills.csv", index=False, encoding="utf-8-sig")

In [ ]:
# 先將技能欄位字串拆分成列表
df["技能清單"] = (
    df["技能"]
    .astype(str)
    .apply(lambda x: [s.strip() for s in x.split(",") if s.strip()])
)

# 將每一列的技能列表展開為多欄
skills_expanded = df["技能清單"].apply(pd.Series)

# 將欄位重新命名為 技能1, 技能2, ...
skills_expanded.columns = [f"技能{i+1}" for i in skills_expanded.columns]

# 合併回原本的 df
df_final = pd.concat([df, skills_expanded], axis=1)

df_final.to_csv("output_skills.csv", index=False, encoding="utf-8-sig")
df_final.filter(regex="^技能")

,技能原文,技能,技能清單,技能1,技能2,技能3,技能4,技能5,技能6,技能7,技能8,技能9,技能10,技能11,技能12,技能13,技能14,技能15,技能16,技能17
0,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...,"HTML,CSS,JavaScript,React,Next.js,Vue,Nuxt.js,...","[HTML, CSS, JavaScript, React, Next.js, Vue, N...",HTML,CSS,JavaScript,React,Next.js,Vue,Nuxt.js,WebXR,Git Flow,Three.js,TypeScript,WebGL,Git,Redux,Node.js,AWS,Github
1,我們致力於建構先進的 VR、AR 和 MR 產品，讓使用者能使用頭盔或瀏覽器享受WebXR體...,"HTML,CSS,JavaScript,TypeScript,React,Next.js,V...","[HTML, CSS, JavaScript, TypeScript, React, Nex...",HTML,CSS,JavaScript,TypeScript,React,Next.js,Vue,Nuxt.js,WebXR,Git Flow,Three.js,WebGL,NaN,NaN,NaN,NaN,NaN
2,工作內容 - 負責 Backend 程式設計並擬定開發計畫 - 檢視並優化服務安全性穩定性...,"Node.js, JavaScript ES6, TypeScript, MongoDB, ...","[Node.js, JavaScript ES6, TypeScript, MongoDB,...",Node.js,JavaScript ES6,TypeScript,MongoDB,Redis,Docker,Distributed Services,HTTP,HTTPS,WebSocket,TCP/IP,DevOps,AWS,GCP,Digital Ocean,GitHub,Git
3,"Yating(https://yating.tw/zh-hant), a Taiwan AI...","TypeScript, JavaScript, Python, Go, PHP, Java,...","[TypeScript, JavaScript, Python, Go, PHP, Java...",TypeScript,JavaScript,Python,Go,PHP,Java,Linux,Ubuntu,Docker,Kubernetes,演算法,資料結構,系統設計,NaN,NaN,NaN,NaN
4,主要職責： • 設計和實施高效能的前端架構。 • 領導前端項目，從規劃到部署，主要使用 Vu...,"HTML,CSS,JavaScript,TypeScript,Vue","[HTML, CSS, JavaScript, TypeScript, Vue]",HTML,CSS,JavaScript,TypeScript,Vue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
